# Preparing the Data

In [58]:
## source: http://www.yelp.com/dataset challenge

## importing packages

import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

### 1. Importing & Converting Data from JSON to CSV

In the Yelp academic dataset that was provided, there are the following files:
    - yelp_academic_dataset_user.json
    - yelp_academic_dataset_tip.json
    - yelp_academic_dataset_review.json
    - yelp_academic_dataset_checkin.json
    - yelp_academic_dataset_business.json
    
For the purposes of this project, I will be using the following, given that I am interested in leveraging the reviews and tips that users have written for certain restaurant establishments:
    - yelp_academic_dataset_tip.json
    - yelp_academic_dataset_review.json
    - yelp_academic_dataset_business.json
    
In the following steps, I will be converting each of those json files to csv, for easier data analysis and manipulation.

In [28]:
# converting business.json into business_data.csv

business_data = {"business_id": [], "name": [], "postal_code": [], "stars": [], "review_count": [], "categories": []}

with open("/Users/avarezvani/Downloads/yelp-dataset/yelp_academic_dataset_business.json") as f:
    for line in tqdm(f):
        business = json.loads(line)
        business_data["business_id"].append(business["business_id"])
        business_data["name"].append(business["name"])
        business_data["postal_code"].append(business["postal_code"])
        business_data["stars"].append(business["stars"])
        business_data["review_count"].append(business["review_count"])
        business_data["categories"].append(business["categories"])

df = pd.DataFrame(business_data)

print(df.shape)
df.head()

df.to_csv("/Users/avarezvani/Downloads/yelp-dataset/business_data.csv", index=False)


192609it [00:02, 70837.03it/s]


(192609, 6)


In [32]:
# converting reviews.json into reviews_data.csv

reviews_data = {"review_id": [], "user_id": [], "business_id": [], "stars": [], "useful": [], "cool": [], "text": [], "date": []}

with open("/Users/avarezvani/Downloads/yelp-dataset/yelp_academic_dataset_review.json") as f:
    for line in tqdm(f):
        reviews = json.loads(line)
        reviews_data["review_id"].append(reviews["review_id"])
        reviews_data["user_id"].append(reviews["user_id"])
        reviews_data["business_id"].append(reviews["business_id"])
        reviews_data["stars"].append(reviews["stars"])
        reviews_data["useful"].append(reviews["useful"])
        reviews_data["cool"].append(reviews["cool"])
        reviews_data["text"].append(reviews["text"])
        reviews_data["date"].append(reviews["date"])

df = pd.DataFrame(reviews_data)

print(df.shape)
df.head()

df.to_csv("/Users/avarezvani/Downloads/yelp-dataset/reviews_data.csv", index=False)


6685900it [00:57, 116817.88it/s]


(6685900, 8)


In [30]:
# converting tips.json into tips_data.csv
# defintion of "tips", per Yelp's website
# "Tips are a way to pass along some key information about a business -- such as the best time to go or your favorite dish 
# -- without writing a full review about your experiences."

tips_data = {"user_id": [], "business_id": [], "text": [], "date": [], "compliment_count": []}

with open("/Users/avarezvani/Downloads/yelp-dataset/yelp_academic_dataset_tip.json") as f:
    for line in tqdm(f):
        tips = json.loads(line)
        tips_data["user_id"].append(tips["user_id"])
        tips_data["business_id"].append(tips["business_id"])
        tips_data["text"].append(tips["text"])
        tips_data["date"].append(tips["date"])
        tips_data["compliment_count"].append(tips["compliment_count"])

df = pd.DataFrame(tips_data)

print(df.shape)
df.head()

df.to_csv("/Users/avarezvani/Downloads/yelp-dataset/tips_data.csv", index=False)



1223094it [00:05, 206552.45it/s]


(1223094, 5)


### 2. Importing New CSV Files & Cleaning to Identify Restaurant Information Only

Starting with the business_data.csv, we are going to narrow down the categories to just restaurants. After that, we will then narrow down the tips_data.csv and the reviews_data.csv to match the business_id's that are leftover in the business_data set.

In [35]:
# business
df_business = pd.read_csv("/Users/avarezvani/Downloads/yelp-dataset/business_data.csv")
df_business.head()

,business_id,name,postal_code,stars,review_count,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,85016,3.0,5,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,L5R 3E7,2.5,128,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,28210,4.0,170,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,85338,5.0,3,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,28217,4.0,4,"Plumbing, Shopping, Local Services, Home Servi..."


In [48]:
# only keep category = "restaurants" in "business"
df_business_restaurants = df_business[df_business["categories"].str.contains("Restaurants", na = False)]

df_business_restaurants.count
print("There are 59,371 businesses that are restaurants")

# checking for any duplicates
df_business_restaurants.business_id.duplicated().sum()
print("There are no duplicates")

There are 59,371 businesses that are restaurants


0

In [50]:
# now let's filter out the reviews that do not match the remaining restaurants
df_reviews = pd.read_csv("/Users/avarezvani/Downloads/yelp-dataset/reviews_data.csv")

df_reviews_restaurants = pd.merge(df_business_restaurants, df_reviews, on = 'business_id')

df_reviews_restaurants.count
print("There are 4,201,684 reviews for these restaurants")

In [56]:
# finally, let's do the same thing we did for reviews, for the tips
df_tips = pd.read_csv("/Users/avarezvani/Downloads/yelp-dataset/tips_data.csv")

df_tips_restaurants = pd.merge(df_business_restaurants, df_tips, on = 'business_id')

df_tips_restaurants.count
print("There are 810,342 tips for these restaurants")

<bound method DataFrame.count of                    business_id                         name postal_code  \
0       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
1       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
2       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
3       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
4       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
5       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
6       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
7       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
8       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
9       QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
10      QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R 3E7   
11      QXAEGFB4oINsVuTFxEYKFQ   Emerald Chinese Restaurant     L5R

### 3. Save those new df's are csv files

In [57]:
df_business_restaurants.to_csv("/Users/avarezvani/Downloads/yelp-dataset/business_restaurants.csv", index=False)
df_reviews_restaurants.to_csv("/Users/avarezvani/Downloads/yelp-dataset/reviews_restaurants.csv", index=False)
df_tips_restaurants.to_csv("/Users/avarezvani/Downloads/yelp-dataset/tips_restaurants.csv", index=False)